In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('/kaggle/input/training-set-minor-sem5/training_set.csv')

In [4]:
df.head()

,Major code stored as workover/drilling,Cde,Service hours type,Minor Repair / Tripping / Complication
0,DDRP,2A,Spud the well Drill Down up to 13 m.,NaN
1,LCON,22C,BACK REAM FROM 1386M TO 880M.,NaN
2,DAUP,11A,ROUND TRIP PRIOR TO VSP IN PROGRESS.,NaN
3,DTRP,6A,UN SEAT THE PACKER.,6A01
4,DTRP,6A,BROKE OFF & L/DN BAKER W/LINE RCI TOOL ASSMBLY,6A02


In [5]:
df.describe()

,Major code stored as workover/drilling,Cde,Service hours type,Minor Repair / Tripping / Complication
count,174890,174890,174890,54231
unique,58,72,129311,51
top,DAUP,6A,POOH.,6A01
freq,33570,40966,1119,34560


In [6]:
df['Service_Hour_Type']=df['Service hours type']

In [7]:
df.head()


,Major code stored as workover/drilling,Cde,Service hours type,Minor Repair / Tripping / Complication,Service_Hour_Type
0,DDRP,2A,Spud the well Drill Down up to 13 m.,NaN,Spud the well Drill Down up to 13 m.
1,LCON,22C,BACK REAM FROM 1386M TO 880M.,NaN,BACK REAM FROM 1386M TO 880M.
2,DAUP,11A,ROUND TRIP PRIOR TO VSP IN PROGRESS.,NaN,ROUND TRIP PRIOR TO VSP IN PROGRESS.
3,DTRP,6A,UN SEAT THE PACKER.,6A01,UN SEAT THE PACKER.
4,DTRP,6A,BROKE OFF & L/DN BAKER W/LINE RCI TOOL ASSMBLY,6A02,BROKE OFF & L/DN BAKER W/LINE RCI TOOL ASSMBLY


In [8]:
df=df.drop('Service hours type', axis=1)

In [9]:
df.head()

,Major code stored as workover/drilling,Cde,Minor Repair / Tripping / Complication,Service_Hour_Type
0,DDRP,2A,NaN,Spud the well Drill Down up to 13 m.
1,LCON,22C,NaN,BACK REAM FROM 1386M TO 880M.
2,DAUP,11A,NaN,ROUND TRIP PRIOR TO VSP IN PROGRESS.
3,DTRP,6A,6A01,UN SEAT THE PACKER.
4,DTRP,6A,6A02,BROKE OFF & L/DN BAKER W/LINE RCI TOOL ASSMBLY


In [10]:
df['Cde'].isnull().sum()

4

In [11]:
df['Service_Hour_Type'].isnull().sum()

4

In [16]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


service_hour_type = df['Service_Hour_Type'].astype(str).tolist()

encoded_inputs=tokenizer(service_hour_type, padding=True, truncation=True, return_tensors='pt', return_attention_mask=True) #padding added to make sure that all sequences are the same length. truncation to make sure the longer tokens are shortened to 512 kb, converts return tensors to pt==pytorch

print("Input IDs:", encoded_inputs['input_ids'])

print("Attention Masks:", encoded_inputs['attention_mask'])







Input IDs: tensor([[  101, 11867,  6784,  ...,     0,     0,     0],
        [  101,  2067,  2128,  ...,     0,     0,     0],
        [  101,  2461,  4440,  ...,     0,     0,     0],
        ...,
        [  101,  1054,  1013,  ...,     0,     0,     0],
        [  101,  8814,  2080,  ...,     0,     0,     0],
        [  101,  1052,  1013,  ...,     0,     0,     0]])
Attention Masks: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [17]:
print(type('Service_Hour_Type'))
print(service_hour_type[:5])

<class 'str'>
['Spud the well Drill Down up to 13 m.', 'BACK REAM FROM 1386M TO 880M.', 'ROUND TRIP PRIOR TO VSP IN PROGRESS.', 'UN SEAT THE PACKER.', 'BROKE  OFF & L/DN  BAKER W/LINE  RCI TOOL ASSMBLY']


In [ ]:
print("Label distribution:", df['CDE'].value_counts())


In [19]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import Trainer, TrainingArguments

#extracting columns
texts=df['Service_Hour_Type'].astype(str).tolist()
labels=df['Cde'].astype('category').cat.codes.tolist()  #converting cde to numeric labels

#tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=len(set(labels)))

#tokenize data
encoded_inputs=tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=128)
input_ids=encoded_inputs['input_ids']
attention_masks=encoded_inputs['attention_mask']
labels=torch.tensor(labels)

#create dataset
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx]
        }

dataset = CustomDataset(input_ids, attention_masks, labels)

# DataLoader
dataloader = DataLoader(dataset, batch_size=64, shuffle=True) #increased batch size from 16 to 64

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1, #changed number of epochs to 1 from 3, just testing for now
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to='wandb',  # Enable W&B logging
    run_name='first_try'  # Optional: specify a run name
)

# Trainer
trainer = Trainer(
    model=model,                         
    args=training_args,                 
    train_dataset=dataset,              
)

# Train the model
trainer.train()


2024-08-20 06:00:46.977183: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 06:00:46.977325: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 06:00:47.173114: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,4.282100
20,4.281200
30,4.251400
40,4.217300
50,4.139300
60,4.005900
70,3.883900
80,3.778700
90,3.641300
100,3.497200


IndexError: Target -1 is out of bounds.